### Exempt Business Master File Region Merging

In [2]:
import pandas as pd

In [5]:
df1 = pd.read_csv('../../data/EBMF/exempt_orgs_gulf_coast.csv')
df2 = pd.read_csv('../../data/EBMF/exempt_orgs_intl.csv')
df3 = pd.read_csv('../../data/EBMF/exempt_orgs_mid_atlantic.csv')
df4 = pd.read_csv('../../data/EBMF/exempt_orgs_puerto_rico.csv')
df5 = pd.read_csv('../../data/EBMF/Regional Giving Data IRS eo1.csv')

In [5]:
embf_merged = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
embf_merged.shape

(1870199, 28)

In [6]:
embf_merged.to_csv('../../data/embf_merged.csv', index=False)


## Joining EIN and other related fields into form 990 data


In [8]:
form990 = pd.read_csv('../../data/22eoextract990.csv')
embf_merged = pd.read_csv('../../data/embf_merged.csv')

form990.shape

(326123, 246)

In [9]:
embf_merged.columns


Index(['EIN', 'NAME', 'ICO', 'STREET', 'CITY', 'STATE', 'ZIP', 'GROUP',
       'SUBSECTION', 'AFFILIATION', 'CLASSIFICATION', 'RULING',
       'DEDUCTIBILITY', 'FOUNDATION', 'ACTIVITY', 'ORGANIZATION', 'STATUS',
       'TAX_PERIOD', 'ASSET_CD', 'INCOME_CD', 'FILING_REQ_CD',
       'PF_FILING_REQ_CD', 'ACCT_PD', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT',
       'NTEE_CD', 'SORT_NAME'],
      dtype='object')

In [13]:
# Perform an inner join
form990_embf = pd.merge(form990,embf_merged[['EIN', 'ACTIVITY','NTEE_CD','SUBSECTION','AFFILIATION']],  # Select only necessary columns from the first DataFrame
                     on='EIN',  
                     how='left')  

# Check the result
print(form990_embf.columns)
print(form990_embf.shape)


Index(['efile', 'EIN', 'tax_pd', 'subseccd', 's501c3or4947a1cd', 'schdbind',
       'politicalactvtscd', 'lbbyingactvtscd', 'subjto6033cd',
       'dnradvisedfundscd',
       ...
       'grsinc509', 'unreltxincls511tx509', 'subtotsuppinc509',
       'netincunrelatd509', 'othrinc509', 'totsupp509', 'ACTIVITY', 'NTEE_CD',
       'SUBSECTION', 'AFFILIATION'],
      dtype='object', length=250)
(326123, 250)


In [14]:
# Standardize column names.
form990_embf.columns = [x.lower() for x in form990_embf.columns]

# Replace zeros with NaN for appropriate columns.

# Replace NaN with appropriate values accordingly.

# Convert columns to appropriate data types.
date_cols = ['tax_pd']
for col in date_cols:
    form990_embf[col] = form990_embf[col].astype(str).str.replace('\.0$', '', regex=True)
    form990_embf[col] = pd.to_datetime(form990_embf[col], format='%Y%m', errors='coerce')

# Drop duplicates by keeping last tax_pd date.
form990_embf = form990_embf.sort_values('tax_pd').drop_duplicates('ein',keep='last') 

# Convert dtype for appropriate columns.
form990_embf['ein'] = form990_embf['ein'].astype(str).str.replace('\.0$', '', regex=True)

form990_embf.head()

,efile,ein,tax_pd,subseccd,s501c3or4947a1cd,schdbind,politicalactvtscd,lbbyingactvtscd,subjto6033cd,dnradvisedfundscd,...,grsinc509,unreltxincls511tx509,subtotsuppinc509,netincunrelatd509,othrinc509,totsupp509,activity,ntee_cd,subsection,affiliation
148177,P,426057254,2011-06-01,2.0,N,N,N,NaN,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,36399000.0,NaN,2.0,3.0
14068,P,60891737,2011-10-01,5.0,N,N,N,NaN,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,232000000.0,K20,5.0,3.0
233203,P,660550623,2011-12-01,3.0,Y,Y,N,N,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,S20,3.0,3.0
168589,P,464039105,2012-06-01,3.0,Y,Y,N,N,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
32657,P,161696098,2012-12-01,3.0,Y,N,N,N,N,N,...,0.0,0.0,0.0,0.0,0.0,121247.0,0.0,T21,3.0,3.0


In [15]:
# Read out the file
form990_embf.to_csv('../../data/form990_embf.csv')